In [14]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget
import h5py
import hdf5plugin
import correction
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc

#Data Files


project_dir = '/data/visitors/balder/20220115/2022022408'
experiment_dir = os.path.join(project_dir, 'raw/XRD Eiger Test')
poni_dir = os.path.join(project_dir, 'process')

stacks = {
    "no_tilt_close": {
        "filename": 'LaB6_9_data_000001.h5',
        "poni": 'samuel_9_0.poni'
    },
    "large_tilt_close": {
       "filename": 'LaB6_10_data_000001.h5',
       "poni": 'samuel_10_0.poni'
    },
    #"small_tilt_close": {
    #    "filename": 'LaB6_11_data_000001.h5',
    #    "poni": 'samuel_11_0.poni'
    #},
    #"no_tilt_far": {
    #    "filename": 'LaB6_13_data_000001.h5',
    #    "poni": 'samuel_13_0.poni'
    #}
}

icsdfilepath = os.path.join(experiment_dir,'LaB6-icsd.txt')
#Parameters
pl = 75  # pixel length, in micrometers 
pw = 450 # pixel width, in micrometers
ph = 75  # pixel height, in micrometers
pixel_dimensions = (pl, pw, ph)
az_npt = 2000 #number of azimuthal integration points
data_layer = 9
radial = (0, 60)

def load_stack(name):
    stack = stacks[name]
    image_path = os.path.join(experiment_dir, stack["filename"])
    with h5py.File(image_path,'r') as h5f:
        data = h5f['/entry/data/data']
        data = np.squeeze(data)
    poni_path = os.path.join(poni_dir, stack["poni"])
    poni = correction.load_poni(poni_path)
    return poni, data

saves = {}
for stack_name in stacks.keys():
    poni, data = load_stack(stack_name)
    img, mask = correction.mask(data[data_layer], False)
    integrator, det_params_b711, tth_hkl = correction.azimutal_fit(img, poni, icsdfilepath, pixel_dimensions)
    recovered_img, G = correction.correct_depth_spill(img, poni, pixel_dimensions, 1)
    integrated_img = integrator.integrate2d(img, az_npt, az_npt, radial_range=radial)[0]
    integrated_recovered = integrator.integrate2d(recovered_img, az_npt, az_npt, radial_range=radial)[0]
    saves[stack_name] = {
        "original_image": img,
        "recovered_image": recovered_img,
        "original_integrated": integrated_img,
        "recovered_integrated": integrated_recovered 
    }
comparisons = {
    # "Full Images": {
    #     "No Tilt - Uncorrected": { "source": "no_tilt_close", "image": "original_image", "area": (slice(0,-1), slice(0,-1)) },
    #     "No Tilt - Corrected": { "source": "no_tilt_close", "image": "recovered_image", "area": (slice(0,-1), slice(0,-1)) },
    #     "Tilt - Uncorrected": { "source": "large_tilt_close", "image": "original_image", "area": (slice(0,-1), slice(0,-1)) },
    # },
    "Full Integrated Images": {
        "No Tilt - Uncorrected": { "source": "no_tilt_close", "image": "original_integrated", "area": (slice(0,-1), slice(0,-1)) },
        "No Tilt - Corrected": { "source": "no_tilt_close", "image": "recovered_integrated", "area": (slice(0,-1), slice(0,-1)) },
        "Tilt - Uncorrected": { "source": "large_tilt_close", "image": "original_integrated", "area": (slice(0,-1), slice(0,-1)) },
    },
    "Compare Orthogonal to Tilted Ground Truth": {
        "No Tilt - Uncorrected": { "source": "no_tilt_close", "image": "original_image", "area": (slice(0,500), slice(200,300)) },
        "No Tilt - Corrected": { "source": "no_tilt_close", "image": "recovered_image", "area": (slice(0,500), slice(200,300)) },
        "Tilt - Uncorrected (close to PONI)": { "source": "large_tilt_close", "image": "original_image", "area": (slice(0,500), slice(0,100)) },
    },
    "Compare Orthogonal to Tilted Ground Truth Zoom Bad": {
        "No Tilt - Uncorrected": { "source": "no_tilt_close", "image": "original_image", "area": (slice(50,150), slice(220,260)) },
        "No Tilt - Corrected": { "source": "no_tilt_close", "image": "recovered_image", "area": (slice(50,150), slice(220,260)) },
        "Tilt - Uncorrected (close to PONI)": { "source": "large_tilt_close", "image": "original_image", "area": (slice(50,150), slice(40,80)) },
    },
    "Compare Orthogonal to Tilted Ground Truth Zoom Good": {
        "No Tilt - Uncorrected": { "source": "no_tilt_close", "image": "original_image", "area": (slice(300,400), slice(200,240)) },
        "No Tilt - Corrected": { "source": "no_tilt_close", "image": "recovered_image", "area": (slice(300,400), slice(200,240)) },
        "Tilt - Uncorrected (close to PONI)": { "source": "large_tilt_close", "image": "original_image", "area": (slice(300,400), slice(20,60)) },
    },
    "Compare Tilted to Orthogonal Ground Truth(ish)": {
        "No Tilt - Uncorrected": { "source": "no_tilt_close", "image": "original_image", "area": (slice(150,500), slice(700,800)) },
        "No Tilt - Corrected": { "source": "no_tilt_close", "image": "recovered_image", "area": (slice(150,500), slice(700,800)) },
        "Tilt - Unorrected": { "source": "large_tilt_close", "image": "original_image", "area": (slice(150,500), slice(550,650)) },
        "Tilt - Corrected": { "source": "large_tilt_close", "image": "recovered_image", "area": (slice(150,500), slice(550,650)) },
    },
    "Integrated Compare Orthogonal to Tilted Ground Truth": {
        "No Tilt - Uncorrected": { "source": "no_tilt_close", "image": "original_integrated", "area": (slice(900*2,-1), slice(480,520)) },
        "No Tilt - Corrected": { "source": "no_tilt_close", "image": "recovered_integrated", "area": (slice(900*2,-1), slice(480,520)) },
        "Tilt - Uncorrected": { "source": "large_tilt_close", "image": "original_integrated", "area": (slice(900*2,-1), slice(480,520)) },
    },
}

#
#
#
#
#
#
#poni, data = load_stack("no_tilt_close")
#img, mask = correction.mask(data[data_layer], False)
#perc = np.percentile(img, 99)
#area = comparisons["Compare Orthogonal to Tilted Ground Truth Zoom Bad"]["No Tilt - Uncorrected"]["area"]

#recovered_img, G = correction.correct_depth_spill(img, poni, pixel_dimensions, 1)
#recovered_local_img, G_loc = correction.correct_depth_spill_local(img, area, poni, pixel_dimensions, 1)
#recovered_up_img = correction.correct_depth_spill_upsampled(img, poni, pixel_dimensions, samp=1, z=2, order=0)

#recovered_img_c, G_c = correction.correct_depth_spill(img, poni, pixel_dimensions, 1, method="carmen")
#recovered_local_img_c, G_loc_c = correction.correct_depth_spill_local(img, area, poni, pixel_dimensions, 10, method="carmen")

#grid_img = np.zeros((img.shape[0] + 2, img.shape[1] + 2))
#grid_img[::10,::10] = 1
#ref_img = (G.get()@grid_img.flatten()).reshape(grid_img.shape)

#plt.imshow(img, vmax=perc)
#plt.figure()
#plt.imshow(recovered_img, vmax=perc)
#plt.figure()
#plt.imshow(recovered_up_img, vmax=perc)

#fig, axs = plt.subplots(2, 3, figsize=(10, 10))
#axs[0,0].imshow(recovered_img[area])
#axs[0,1].imshow(recovered_local_img)
#axs[0,2].imshow(img[area])
#axs[1,0].imshow(recovered_up_img[area])
#axs[1,1].imshow(grid_img[area])
#axs[1,2].imshow(ref_img[area])

perc = np.percentile(saves["no_tilt_close"]["original_image"], 99)

for comp_name, comparison in comparisons.items():
   n = len(comparison)
   fig, axs = plt.subplots(1, n, figsize=(10, 10), sharex=True, sharey=True)
   fig.suptitle(comp_name)
   for i, (name, info) in enumerate(comparison.items()):
        im = saves[info["source"]][info["image"]][info["area"]]
        axs[i].imshow(im, vmax=perc)
        axs[i].set_title(name)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …